In [151]:
# import statements
import requests
import re
import json
import time
from bs4 import BeautifulSoup as BS

In [152]:
def scrape(url):
    # need header to scrape from wikipedia
    headers = {
        'User-Agent': 'SchoolProjectBot/1.0 (r1043412@student.thomasmore.be) Python-Requests/2.31.0'
    }
    page = requests.get(url, headers=headers)
    
    time.sleep(2)
    
    return BS(page.text, "html")

In [153]:
def write_json(dict):
    with open(f'data/rider_teams.json', 'r+') as file:
        # load the entire json file into memory
        file_json = json.load(file)

        # update the json with the python dictionary
        file_json['rider'].append(dict)

        # put cursor at beginning of file and overwrite it with the dictionary in memory
        file.seek(0)
        json.dump(file_json, file, indent=4)

In [154]:
def get_team_links():
    soup = scrape('https://www.procyclingstats.com/race/tour-de-france')

    # debug
    # print(soup.find('ul', class_='topnav'))

    team_links = [
        'https://www.procyclingstats.com/' +  team.find('a')['href']

        for team in soup.find('ul', class_='topnav').select('li')[19].find('ul').select('li')
    ]

    return team_links

In [155]:
def scrape_team(team_url):
    # scrape the team page first
    soup = scrape(team_url)

    # extract all rider names and links
    # why class_ and not class? TODO
    riders = [
        {'name': rider.find('div', class_='name').get_text(strip=True), 'link': rider.find('a')['href']}

        for rider in soup.find('ul', class_='photos').select('li')
    ]

    # rider
    # print(riders)

    for i, rider in enumerate(riders, start=0):
        soup = scrape('https://www.procyclingstats.com/' + rider['link'])

        # debug
        print(soup.find('ul', class_='rdr-teams2').select('li'))

        rider_teams = [
            team.find('div', class_='name').get_text(strip=True)
            
            for team in soup.find('ul', class_='rdr-teams2').select('li')

            if team.find('div', class_='name')
        ]

        riders[i].update({'teams': rider_teams})

    write_json(riders)

In [156]:
links = get_team_links()

[scrape_team(link) for link in links]

[<li class="main"><div class="season">2026</div><div class="name"><a href="team/xds-astana-team-2026">XDS Astana Team</a> (WT)<span class="fs10"></span></div><div class="clear"></div></li>, <li class="main"><div class="season">2025</div><div class="name"><a href="team/xds-astana-team-2025">XDS Astana Team</a> (WT)<span class="fs10"></span></div><div class="clear"></div></li>, <li class="main"><div class="season">2024</div><div class="name"><a href="team/uae-team-emirates-2024">UAE Team Emirates</a> (WT)<span class="fs10"></span></div><div class="clear"></div></li>, <li class="main"><div class="season">2023</div><div class="name"><a href="team/uae-team-emirates-2023">UAE Team Emirates</a> (WT)<span class="fs10"></span></div><div class="clear"></div></li>, <li class="main old"><div class="season">2022</div><div class="name"><a href="team/uae-team-emirates-2022">UAE Team Emirates</a> (WT)<span class="fs10"></span></div><div class="clear"></div></li>, <li class="main old"><div class="seaso

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]